#### Setup for file

In [3]:
import pandas as pd
import numpy as np
import streamlit as st
import openai
import time
from bs4 import BeautifulSoup
import requests
import re
import os
import json

from openai._client import OpenAI
from openai import AzureOpenAI


client = AzureOpenAI(
            azure_endpoint="https://da-openai-test.openai.azure.com/",
            api_key="5a90b85ba420469c9b36438e238d70fe",
            api_version="2023-05-15",
        )


In [4]:

today = time.strftime("%m_%d")
today_hour = time.strftime("%m_%d_%H")

# Make a folder for today"s data inside the data folder
if not os.path.exists("data_2024/" + today):
    os.mkdir("data_2024/" + today)


In [5]:

# file_path_original_dataframe_csv = "data/all_products_block123.csv"
file_path_original_dataframe_excel = "data_2024/All Products (9).xlsx"
# exclude the following columns : product_tags from the original dataframe
columns_to_exclude = ["product_tags"]



# file_path_filter_dataframe_csv = "filter_tags.csv"
file_path_filter_dataframe_excel = "data_2024/Filter Tags.xlsx"

new_file_path_splitted_dataframe_excel = "data_2024/"+today+"/splitted_tags_" + today_hour + ".xlsx"
new_file_path_splitted_dataframe_csv = "data_2024/"+today+"/splitted_tags_" + today_hour + ".csv"

new_file_path_grouped_dataframe_excel = "data_2024/"+today+"/grouped_tags_" + today_hour + ".xlsx"
new_file_path_grouped_dataframe_csv = "data_2024/"+today+"/grouped_tags_" + today_hour + ".csv"

In [107]:
# # load the dict from pickle file
# import pickle
# with open("./data/url_technical_text_dict.pkl", "rb") as handle:
#     url_text_dict = pickle.load(handle)

#### Explode the dataframe

In [6]:
df = pd.read_excel(file_path_original_dataframe_excel)
df = df.drop(columns=columns_to_exclude)
print(len(df))

# Split the "Product category" column into multiple rows
df = df.assign(
    **{"Product category": df["Product category"].str.split(",")}
).explode("Product category")

# Trim whitespace from the "Product category" column
df["Product category"] = df["Product category"].str.strip()


print(len(df))



261
402


In [10]:
df

,Unnamed: 0,Product_Name,Product category,Introduction,Block1,Block2,Block3,Specifications,Features,url,Data sheets,downloads,Hero Image,Other Image,is_software,grouped_data_sheets,Unnamed: 17
0,0,"Seapath 380 - utilising GPS, GLONASS, Galileo...",Navigation & positioning,"{""heading"": ""Introduction"", ""text"": ""The Seapa...","{""heading"": ""Advanced Navigation Integration"",...","{""heading"": ""Functional Design and System Conf...","{""heading"": ""Product Range and Applications"", ...","[{""Title"": ""Performance Power Specifications"",...",0.007° to 0.02° roll and pitch accuracy depend...,NaN,https://www.kongsberg.com/contentassets/c22a59...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",[],[],False,NaN,NaN
1,1,12 kHz dual-beam transducer,Ocean science,"{""heading"": ""Introduction"", ""text"": ""The 12-16...",NaN,NaN,NaN,"[{""Title"": ""Transducer Characteristics"", ""Spec...",NaN,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",['https://www.kongsberg.com/contentassets/e811...,[],False,"{""productRange"": [""38 kHz Single-beam transduc...",NaN
2,2,120 kHz single-beam transducer,Ocean science,"{""heading"": ""Introduction"", ""text"": ""The 120-2...",NaN,NaN,NaN,"[{""Title"": ""Acoustic Properties"", ""Specificati...",NaN,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",['https://www.kongsberg.com/contentassets/e811...,[],False,"{""productRange"": [""38 kHz Single-beam transduc...",NaN
3,3,18 kHz single-beam transducer,Ocean science,"{""heading"": ""Introduction"", ""text"": ""The 18-11...",NaN,NaN,NaN,"[{""Title"": ""Frequency and Response"", ""Specific...",NaN,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",['https://www.kongsberg.com/contentassets/e811...,[],False,"{""productRange"": [""38 kHz Single-beam transduc...",NaN
4,4,200 kHz single-beam transducer in robust housing,Ocean science,"{""heading"": ""Introduction"", ""text"": ""The 200-2...",NaN,NaN,NaN,"[{""Title"": ""Acoustic characteristics"", ""Specif...",NaN,https://www.kongsberg.com/maritime/products/oc...,https://www.kongsberg.com/contentassets/d106c9...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",['https://www.kongsberg.com/contentassets/e811...,[],False,"{""productRange"": [""38 kHz Single-beam transduc...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,271,XPR 100,Navigation & positioning,"{""heading"": ""Introduction"", ""text"": ""The XPR 1...","{""heading"": ""Optimized for Dynamic Positioning...","{""heading"": ""Advanced Targeting and Verificati...","{""heading"": ""Certification and Complementary T...","[{""Title"": ""Range Specifications"", ""Specificat...",No moving parts#\nNo regular maintenance#\nCom...,https://www.kongsberg.com/maritime/products/ve...,https://www.kongsberg.com/contentassets/ebe8c6...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",['https://www.kongsberg.com/contentassets/8941...,['https://www.kongsberg.com/contentassets/8941...,False,"{""productRange"": [""XPR 100 EX LP"", ""XPR 100"", ...",NaN
257,272,XPR 100 EX,Navigation & positioning,"{""heading"": ""Introduction"", ""text"": ""The XPR 1...","{""heading"": ""Optimized for Dynamic Positioning...","{""heading"": ""Advanced Targeting and Verificati...","{""heading"": ""Certification and Complementary T...","[\n {\n ""Title"": ""Environmental Specificat...",No moving parts#\nNo regular maintenance#\nCom...,https://www.kongsberg.com/maritime/products/ve...,https://www.kongsberg.com/contentassets/ebe8c6...,"{""Heading"": ""Downloads"", ""Sections"": [{""Headin...",['https://www.kongsberg.com/contentassets/8941...,['https://www.kongsberg.com/contentassets/8941...,False,"{""product

#### Make new products_tags

In [110]:
print()

In [7]:
df.to_csv(new_file_path_splitted_dataframe_csv, index=False)
df.to_excel(new_file_path_splitted_dataframe_excel, index=False)

In [8]:
import signal

# Define your timeout handler
def handler(signum, frame):
    raise Exception("Operation timed out")
signal.signal(signal.SIGALRM, handler)



<Handlers.SIG_DFL: 0>

In [10]:
# Create a new column to store the tags for each product
# df_tech_new_tags = pd.read_csv("./data/tags_30_11.csv", usecols=["Product_Name", "Product category", "url", "image_url"])
df_tech_new_tags = pd.read_csv(new_file_path_splitted_dataframe_csv)
filter_dataframe = pd.read_excel(file_path_filter_dataframe_excel)

# Set "category" as the index of filter_dataframe for easier lookup
filter_dataframe.set_index("category", inplace=True)

# Iterate over the rows of the dataframe
for index, row in df_tech_new_tags.iterrows():
    if pd.isna(row["product_tags"]):
        print(index)
        product_category = str(row["Product category"])  # Convert to string
        # Get the tags for the product category
        if product_category in filter_dataframe.index:
            category_tags = filter_dataframe.loc[product_category].to_dict()  # Use a different variable
            print(row["Product_Name"])
            print(row["Product category"])
            if not pd.isna(row["Data sheets"]):
                print(row["Data sheets"])
            print(category_tags)
            intro_text = str(row["Introduction"]) if pd.notna(row["Introduction"]) else ""
            block1 = str(row["Block1"]) if pd.notna(row["Block1"]) else ""
            block2 = str(row["Block2"]) if pd.notna(row["Block2"]) else ""
            block3 = str(row["Block3"]) if pd.notna(row["Block3"]) else ""
            Features = str(row["Features"]) if pd.notna(row["Features"]) else ""
            Technical_specifications = str(row["Specifications"]) if pd.notna(row["Specifications"]) else ""

            # Concatenate all the text
            website_text = intro_text + block1 + block2 + block3 + Features + Technical_specifications
            messages = [
                {
                    "role": "system",
                    "content": """You are a helpful assistant that generates relevant tags for products. Your responses should be in json format with the top-level category as the key and the tags as the value. Always use " and not '. For example:
                    
                    {{
                        "Area": "tag", "tag", "tag",
                        "Product Type": "tag", "tag", "tag",
                        "Technology": "tag", "tag", "tag",
                        "Application": "tag", "tag","tag",
                    }}
                    """
                },
                {
                    "role": "user",
                    "content": f"""For the product named "{row["Product_Name"]}" in the category " {row["Product category"]} " with the following description: "{website_text}", please select as many tags that is correct from each top-level category from the provided list: {category_tags}. The top-level categories include "Area", "Product Type", "Technology", and "Application". The maximum total number of tags is 15, but you are not required to use all 15. Your response should be a comma-separated list of tags. Please note: avoid using "tag" as a tag, and do not use the top-level category names as tags. There could be multiple tags from the same top-level category. Return the tags in json format withthe top-level category as the key and the tags as the value. (Always use " and not '). For example:
                    
                    {{
                        "Area": "tag", "tag", "tag",
                        "Product Type": "tag", "tag", "tag",
                        "Technology": "tag", "tag", "tag",
                        "Application": "tag", "tag","tag",
                
                    }}
                    """
                }
            ]
            # response = client.chat.completions.create(
            #     model="gpt-4-1106-preview",
            #     messages=messages
            #     # response_format={ "type": "json_object" }
            # )
            
           
            success = False
            while not success:
                try:
                    signal.alarm(2)  # Set a 120-second alarm
                    response = client.chat.completions.create(
                        model="gpt-4-turbo-1106-preview",
                        messages=messages,
                        response_format={ "type": "json_object" }
                    )
                    signal.alarm(0)  # Disable the alarm

                    tags = response.choices[0].message.content.strip()
                    df_tech_new_tags.loc[index, "product_tags"] = tags
                    tags_json = json.loads(tags)
                    data = {row["Product category"]: tags_json}
                    success = True
                    print("______________")
                    print(data)
                    print("______________")
                    df_tech_new_tags.loc[index, "product_tags"] = json.dumps(data)
                except Exception as e:
                    if str(e) == "Operation timed out":
                        print("Operation timed out, skipping...")
                        break
                    else:
                        print("JSONDecodeError occurred, retrying...")

        else:
            print(f"""Category "{product_category}" not found in filter_dataframe""")
        df_tech_new_tags.to_csv(new_file_path_splitted_dataframe_csv, index=False)
        df_tech_new_tags.to_excel(new_file_path_splitted_dataframe_excel, index=False)

# save the dataframe as csv
df_tech_new_tags.to_csv(new_file_path_splitted_dataframe_csv, index=False)
df_tech_new_tags.to_excel(new_file_path_splitted_dataframe_excel, index=False)

0
 Seapath 380 - utilising GPS, GLONASS, Galileo, Beidou and QZSS
Navigation & positioning
https://www.kongsberg.com/contentassets/c22a596095db4994a5f26ba216b6b968/110-0017978a_datasheet_seapath380_dec22.pdf
{'Area': 'Subsea, Surface, Offshore, Onshore', 'product type': 'GNSS reciever, IALA beacon reciever, Inertial measurement unit (IMU), Echo sounder, Hydroacoustics (HIPAP), Relative reference system, Absolute reference systems, Transponder, Prism', 'technology': 'GNSS infrastructure, Sensor fusion, Spoofing mitigation, Augmented reality, Acoustic telemetry, Hydroacoustic positioning, Inertial measurement unit, Laser, Microwave', 'application': 'Situational awareness, GNSS, Navigation, Vessel tracking, Asset tracking, Offshore loading, Relative positioning, Absolute positioning, DP reference, Seismic'}


/var/folders/3c/7nbqh9x54rsgdpqc1dpcxh7w0000gn/T/ipykernel_90640/4109995367.py:77: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '{
    "Area": "Subsea, Surface, Offshore, Onshore",
    "Product Type": "GNSS receiver, Inertial measurement unit (IMU)",
    "Technology": "GNSS infrastructure, Sensor fusion, Inertial measurement unit",
    "Application": "GNSS, Navigation, Asset tracking, DP reference, Seismic"
}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_tech_new_tags.loc[index, "product_tags"] = tags


______________
{'Navigation & positioning': {'Area': 'Subsea, Surface, Offshore, Onshore', 'Product Type': 'GNSS receiver, Inertial measurement unit (IMU)', 'Technology': 'GNSS infrastructure, Sensor fusion, Inertial measurement unit', 'Application': 'GNSS, Navigation, Asset tracking, DP reference, Seismic'}}
______________
1
12 kHz dual-beam transducer
Ocean science
https://www.kongsberg.com/contentassets/d106c9aa05de423a8b02cf5fae6d6ad1/160920_transducer_12-16_60.pdf
{'Area': nan, 'product type': 'Scientific Echosounders,  Scientific multibeam systems, ADCP (Acoustic Doppler current profiler)', 'technology': 'Wideband technology, Acoustic Doppler, GPS integrated system, Split-beam transducer, Autonomous transceiver, Sonar technology, Transducer', 'application': 'Scientific research, Environmental monitoring, Fish stock assessment, Marine navigation'}
______________
{'Ocean science': {'Area': 'Ocean science', 'Product Type': 'Scientific Echosounders', 'Technology': 'Sonar technology, 

#### Groupe the dataframe

In [ ]:
import ast
def extract_tags(tags_str):
    tags_obj = ast.literal_eval(tags_str)
    if isinstance(tags_obj, dict):
        tags_values = tags_obj.values()
    elif isinstance(tags_obj, tuple):
        tags_values = tags_obj
    else:
        raise ValueError(f"Unexpected type {type(tags_obj)}: {tags_obj}")
    
    tags_list = []
    for value in tags_values:
        if isinstance(value, list):
            tags_list.append(", ".join(map(str.strip, value)))
        elif isinstance(value, dict):
            tags_list.append(str(value))
        else:
            tags_list.append(value.strip())
    return ", ".join(tags_list)

In [ ]:
df_with_many_tags = pd.read_csv(new_file_path_splitted_dataframe_csv)
print(len(df_with_many_tags))
# df_with_many_tags["product_tags"] = df_with_many_tags["product_tags"].apply(extract_tags)
# combine all the tags with same product_name into one row. Let the name be the same as the first row, but for tags, combine them into two lists of tags. For the category, combine them into a list of categories. For website_url, image_url, choose the first one. 



df_with_many_tags = df_with_many_tags.groupby("Product_Name").agg({
    "product_tags": lambda x: ", ".join(set(map(lambda y: str(y).strip(), x))),
    "Product category": lambda x: ", ".join(map(lambda y: str(y).strip(), x)),
    "url": "first",
    "Data sheets": "first",
    "Introduction": "first",
    "Block1": "first",
    "Block2": "first",
    "Block3": "first",
    "Features": "first",
    "Technical Specifications": "first",
    "downloads": "first",
    "product_family_name": "first",
    "is_range": "first",
    "Hero Image": "first",

    
    


    
}).reset_index()
print(len(df_with_many_tags))

# save to excel
df_with_many_tags.to_excel(new_file_path_grouped_dataframe_excel, index=False)
df_with_many_tags.to_csv(new_file_path_grouped_dataframe_csv, index=False)

444
262


#### Merge with software tags

In [ ]:
# # if dataframe do not have column "is_software" then merge the column from software dataframe to the dataframe on the product name column
# df_grouped_tags = pd.read_csv(new_file_path_grouped_dataframe_csv)
# #check if the column is_software exists
# if "is_software" not in df_grouped_tags.columns:
#     # read the software dataframe
#     df_software = pd.read_csv("data/tags_27_11.csv")
#     # merge the column is_software to the dataframe on the product name column
#     df_grouped_tags = pd.merge(df_grouped_tags, df_software[["Product_Name", "is_software"]], on="Product_Name", how="left")
#     # save the dataframe
#     print(df_grouped_tags.head())

                                        Product_Name  \
0                        AIS physical shore stations   
1                      AIS service management system   
2                                AIS space receivers   
3  APOS - Operating station for HiPAP and HPR Aco...   
4  APOS Survey - Surveyor's independent Operator ...   

                                        product_tags  \
0  AIS transponder, Autonomous Navigation, Underw...   
1  Acoustic Control System, AIS transponder, Mari...   
2  Transceiver, Radio, reconfigurable SDR, VDES t...   
3  Navigation system, Positioning system, Acousti...   
4  Navigation system, Positioning system, Acousti...   

                      Product category  \
0            Surveillance & monitoring   
1            Surveillance & monitoring   
2              Communication solutions   
3  Underwater navigation & positioning   
4  Underwater navigation & positioning   

                                                 url  \
0  https://www.ko